In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Thesis project/Thesis/BERTmodels'

/content/drive/MyDrive/Thesis project/Thesis/BERTmodels


In [3]:
%%capture
!pip install pytorch-pretrained-bert
!pip install transformers
!pip install datasets
!pip install emoji

In [4]:
from  bertModel import *

import os
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

import pandas as pd
import numpy as np
import csv

In [5]:
models_path="/content/drive/MyDrive/Thesis project/Thesis/BERTmodels/savedModels/selectedModels/"
files_lst = os.listdir(models_path)


absolute_datasets_path="/content/drive/MyDrive/Thesis project/Thesis/datasets/"



#datasets="SARC_hot_topics_train.csv SARC_balanced_topics_train.csv
datasets="SARC_hot_topics_test.csv SARC_balanced_topics_test.csv SARC_random_test.csv preprocess_semEval2022_test.csv"

# Loading multiple models

In [6]:
#load the pre-trained models from specific path
# and evaluate them to receive their results

models_lst=[]
for f in files_lst:
  models_lst.append(models_path+f)

In [7]:
print(models_lst)

['/content/drive/MyDrive/Thesis project/Thesis/BERTmodels/savedModels/selectedModels/houmorBERT', '/content/drive/MyDrive/Thesis project/Thesis/BERTmodels/savedModels/selectedModels/toxcity_datasetBERT', '/content/drive/MyDrive/Thesis project/Thesis/BERTmodels/savedModels/selectedModels/irony_trainBERT']


## f1 scores

In [8]:
# models_names=[]
# for model in models_lst:
#   models_names.append(model.split('/')[-1])

# rows=[]

# header=['model name']
# ds_lst=datasets.split(' ')
# for ds in ds_lst:
#   header.append(ds)

# for i in range(len(models_lst)):
#   b=BERTModel(model_name=models_lst[i],
#                   absolute_path=absolute_datasets_path,
#                   datasets=datasets,
#                   mode="testing",
#                   print_res=False)

#   model_name=b.get_model_name()
#   b.evaluate_model()
#   cr=b.get_classifiction_report()
#   keys=list(cr.keys()) # list of test datasets

#   row=[b.get_model_name()]
#   for i in range(len(keys)):
#     res=cr[keys[i]]
#     test_dataset=keys[i].split('/')[-1]
#     try:
#       f1_pos=round(res['1']['f1-score'],3)
#     except:
#       f1_pos=" "
#     try:
#       f1_neg=round(res['0']['f1-score'],3)
#     except:
#       f1_neg= ""

#     acc=round(res['accuracy'],3)
#     macro_f1=round(res['macro avg']['f1-score'],3)
#     weighted_macro=round(res['weighted avg']['f1-score'],3)

#     row.append(f1_pos)
#   rows.append(row)

# with open("sarcasem_detection_model_results.csv", 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)


#     #write multiple rows
#     writer.writerows(rows)


## Prediction mission

loading multiple model and for each one of the models we use him to predict every sentence and save the preditction of each sentences in table cell

In [9]:


# by defualt we cheak only one dataset
ds=absolute_datasets_path+"SARC_hot_topics_train.csv"
#ds=absolute_datasets_path+"SARC_60K_comments.csv"
#ds=absolute_datasets_path+"60K_RoastMe_comments.csv"


models_names=[]
for model in models_lst:
  models_names.append(model.split('/')[-1])

header=['text','label']
for model in models_names:
  header.append(model)
header.append('sum_of_probs')
header.append("product_of_probs")




models_dic=dict()
for model in models_names:
  models_dic[model]=[]

with open("BERTmodel_results_on_SARC_hot_topics", 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  rows=[]
  data = pd.read_csv(ds)
  sentences=data['text']
  labels=data['label']
  # write the header
  writer.writerow(header)
  for i in range(len(models_lst)):
    b=BERTModel(model_name=models_lst[i],
                    absolute_path=absolute_datasets_path,
                    datasets=datasets,
                    mode="testing",
                    print_res=False)

    model_name=b.get_model_name()
    for i in range(len(sentences)):
      if (sentences[i] is not None and sentences[i]!=""):
        res=b.predict(sentences[i])
        probs=res[1]
        pos_probs=probs[0][1]
        models_dic[model_name].append(pos_probs)

  #save the results in csv file
  for i in range(len(sentences)):
    sum_probs=0
    prod_probs=1
    row=[sentences[i],labels[i]]
    for model in models_dic:
      prob=models_dic[model][i]
      row.append(prob)
      sum_probs+=prob
      prod_probs=prod_probs*prob

    row.append(sum_probs)
    row.append(prod_probs)
    rows.append(row)

  #write multiple rows
  writer.writerows(rows)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s


# Load one model

### set the dataset path and the titles of the csv result table

here we chosse one the models from the saved model list and store him at bert_model

In [10]:
ds_lst=datasets.split(" ")
full_path_datasets=[]
for i in range(len(ds_lst)):
  full_path_datasets.append(absolute_datasets_path+ds_lst[i])


header=['model name','sentence','label','probability']

bert_model=models_lst[0]
print(bert_model)

b=BERTModel(model_name=bert_model,
                      absolute_path=absolute_datasets_path,
                      datasets=datasets,
                      mode="testing",
                      print_res=False
                      )

b.evaluate_model()

/content/drive/MyDrive/Thesis project/Thesis/BERTmodels/savedModels/selectedModels/houmorBERT
testing


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


testing
testing
testing


### save  f1 results in csv files

In [11]:

# rows=[]
# header=['model name','training dataset','test dataset','F1-positive class','F1- negative class','accuracy','macro avg','weighted avg']

# cr=b.get_classifiction_report()
# keys=list(cr.keys()) # list of test datasets

# with open('sarcasem_detection_model_results'+"_"+b.get_model_name()+".csv", 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)

#     for i in range(len(keys)):
#       res=cr[keys[i]]
#       test_dataset=keys[i].split('/')[-1]
#       try:
#         f1_pos=round(res['1']['f1-score'],3)
#       except:
#         f1_pos=" "
#       try:
#         f1_neg=round(res['0']['f1-score'],3)
#       except:
#         f1_neg= ""


#       acc=round(res['accuracy'],3)
#       macro_f1=round(res['macro avg']['f1-score'],3)
#       weighted_macro=round(res['weighted avg']['f1-score'],3)

#       row=[b.get_model_name(),'',test_dataset, f1_pos,f1_neg,acc, macro_f1,weighted_macro]
#       rows.append(row)
#     #write multiple rows
#     writer.writerows(rows)


### print f1 results

In [12]:
# cr=b.get_classifiction_report()
# keys=list(cr.keys()) # list of test datasets
# for i in range(len(keys)):
#   res=cr[keys[i]]
#   test_dataset=keys[i].split('/')[-1]
#   f1_neg=res['0']['f1-score']
#   f1_pos=res['1']['f1-score']
#   acc=res['accuracy']
#   print(test_dataset)
#   print(round(f1_pos,3),round(f1_neg,3),round(acc,3),sep="\t")

### preditcion

In [13]:
# sent="Can I get ready for work in 10?Can I get ready for work in 10? #late"
# res=b.predict(sent)
# prediction=res[0]
# probs=res[1]
# pos_probs=probs[0][1]

### save positive and negative results in csv files

read sentences from test dataset and save the  porobility for
each sentence.

negative probilities

In [14]:
# for i in range(len(full_path_datasets)):
#   rows=[]
#   data = pd.read_csv(full_path_datasets[i])
#   sentences=data['text']
#   labels=data['label']

#   with open(b.get_model_name()+"_"+"negative_probilities"+"_"+ds_lst[i], 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)


#     # write the header
#     writer.writerow(header)
#     for i in range(len(sentences)):
#       res=b.predict(sentences[i])
#       probs=res[1]
#       neg_probs=probs[0][0]
#       row=[b.get_model_name(),sentences[i],labels[i],neg_probs]
#       rows.append(row)
#     #write multiple rows
#     writer.writerows(rows)


positive probilities

In [15]:
# for i in range(len(full_path_datasets)):
#     rows=[]
#     data = pd.read_csv(full_path_datasets[i])
#     sentences=data['text']
#     labels=data['label']
#     with open(b.get_model_name()+"_"+"positive_probilities"+"_"+ds_lst[i], 'w', encoding='UTF8', newline='') as f:
#       writer = csv.writer(f)
#       # write the header
#       writer.writerow(header)
#       for i in range(len(sentences)):
#         res=b.predict(sentences[i])
#         probs=res[1]
#         pos_probs=probs[0][1]
#         # if (pos_probs>=0.8):
#         #   row=[b.get_model_name(),sentences[i],labels[i],pos_probs]
#         row=[b.get_model_name(),sentences[i],labels[i],pos_probs]
#         rows.append(row)
#       #write multiple rows
#       writer.writerows(rows)

### plot historgrams

plot  2 histograms of the current model for each datasets.
one for positive labels and the other for negative labels

In [16]:
# # n=array_of_bins_counts
# def plot_values_on_top_of_bins_bars(bins,n):

#   # x ticks is the position of the values: on the center of each one of the bins
#     xticks = [(bins[idx+1] + value)/2 for idx, value in enumerate(bins[:-1])]

#     # plot values on top of bars
#     for idx, value in enumerate(n):
#         if value > 0:
#             #t=int(value*100)
#             t=round(value*100)
#             s=str(t) +"%"
#             plt.text(xticks[idx], value+0.001, s, ha='center')



# for i in range(len(full_path_datasets)):
#   dataset_name=(full_path_datasets[i].split('/')[-1]).split('.')[0]
#   print(dataset_name)

#   data = pd.read_csv(full_path_datasets[i])
#   data= data.loc[:,["text","label"]] # get the relveant cloumns from the dataset
#   data=data.sort_values('label') # sort the values by the label
#   data_neg_labels = data[data['label'] <=0]
#   data_pos_labels = data[data['label'] >=1]

#   data_labels_lst=[data_neg_labels,data_pos_labels]
#   for label_idx in range(len(data_labels_lst)):# there are two labels: positive and negative
#     probs_lst=[]
#     # go over all the sentences and predict each one
#     for j in range(len(data_labels_lst[label_idx])):
#       text=data_labels_lst[label_idx].iloc[j]['text']
#       res=b.predict(text)
#       probs=res[1]
#       pos_probs=probs[0][1]
#       probs_lst.append(pos_probs)


#     plt.figure(figsize=(4,4)) #change your figure size as per your desire here
#     n, bins, patches =plt.hist(probs_lst, bins=[0,0.25,0.5,0.75,1],weights=np.ones(len(probs_lst)) / len(probs_lst),rwidth=0.95,alpha=0.7)
#     plt.xlabel('probability')
#     plt.ylabel('percentages')
#     model_name=b.get_model_name()
#     kind_of_labels=""
#     if(label_idx==0):kind_of_labels="negative"
#     else: kind_of_labels="positive"
#     text=model_name+" on "+ dataset_name+ " "+kind_of_labels+ " labels probs "
#     #plt.title(text)
#     print(text)
#     print("")

#     plot_values_on_top_of_bins_bars(bins,n)
#     plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
#     plt.show()

